In [1]:
import netsquid as ns

## Qubits

In [2]:
qubits = ns.qubits.create_qubits(1)
qubits
qubit = qubits[0]
# To check the state is |0> we check its density matrix using reduced_dm():
ns.qubits.reduced_dm(qubit)

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j]])

In [3]:
# transform qubit state from 0 to 1 using X operator
ns.qubits.operate(qubit, ns.X)
ns.qubits.reduced_dm(qubit)

array([[0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])

In [5]:
# measure using standard basis (Z)
measurement_result, prob = ns.qubits.measure(qubit)
if measurement_result == 0:
    state = "|0>"
else:
    state = "|1>"
print(f"Measured {state} with probability {prob:.1f}")

Measured |1> with probability 1.0


In [9]:
# measure using Hadamard basis (X) to random outcome
measurement_result, prob = ns.qubits.measure(qubit, observable=ns.X)
if measurement_result == 0:
    state = "|+>"
else:
    state = "|->"
print(f"Measured {state} with probability {prob:.1f}")
ns.qubits.reduced_dm(qubit)

Measured |+> with probability 1.0


array([[0.5+0.j, 0.5+0.j],
       [0.5+0.j, 0.5+0.j]])

## Simulation Engine

#### to track time evolution of qubit quantum states and account for communication and processing delays in a network

## Components

In [10]:
# create two remote nodes
from netsquid.nodes import Node
node_ping = Node(name="Ping")
node_pong = Node(name="Pong")

In [11]:
# create transmission delay model that depeends on lenth of each channel
# avg qubit speed = c/2, with SD = .05
from netsquid.components.models import DelayModel

class PingPongDelayModel(DelayModel):
    def __init__(self, speed_of_light_fraction=0.5, standard_deviation=0.05):
        super().__init__()
        # (the speed of light is about 300,000 km/s)
        self.properties["speed"] = speed_of_light_fraction * 3e5
        self.properties["std"] = standard_deviation
        self.required_properties = ['length']  # in km

    def generate_delay(self, **kwargs):
        avg_speed = self.properties["speed"]
        std = self.properties["std"]
        # The 'rng' property contains a random number generator
        # We can use that to generate a random speed
        speed = self.properties["rng"].normal(avg_speed, avg_speed * std)
        delay = 1e9 * kwargs['length'] / speed  # in nanoseconds
        return delay

In [12]:
# create quantum channels (l=2.74 m)
from netsquid.components import QuantumChannel

distance = 2.74 / 1000  # default unit of length in channels is km
delay_model = PingPongDelayModel()
channel_1 = QuantumChannel(name="qchannel[ping to pong]",
                           length=distance,
                           models={"delay_model": delay_model})
channel_2 = QuantumChannel(name="qchannel[pong to ping]",
                           length=distance,
                           models={"delay_model": delay_model})

In [13]:
# create connection component to wrap channels into one component
# so Ping and Pong can send/receive qubits to/from their qubitIO ports
from netsquid.nodes import DirectConnection

connection = DirectConnection(name="conn[ping|pong]",
                              channel_AtoB=channel_1,
                              channel_BtoA=channel_2)
node_ping.connect_to(remote_node=node_pong, connection=connection,
                     local_port_name="qubitIO", remote_port_name="qubitIO")

('qubitIO', 'qubitIO')

## Protocols

####  protocol we need at each node should wait for any incoming qubits on the node’s port. When a qubit arrives it should measure it in the preferred basis, then directly send it back through the same port. 

In [14]:
from netsquid.protocols import NodeProtocol

class PingPongProtocol(NodeProtocol):
    def __init__(self, node, observable, qubit=None):
        super().__init__(node)
        self.observable = observable
        self.qubit = qubit
        # Define matching pair of strings for pretty printing of basis states:
        self.basis = ["|0>", "|1>"] if observable == ns.Z else ["|+>", "|->"]

    def run(self):
        if self.qubit is not None:
            # Send (TX) qubit to the other node via port's output:
            self.node.ports["qubitIO"].tx_output(self.qubit)
        while True:
            # Wait (yield) until input has arrived on our port:
            yield self.await_port_input(self.node.ports["qubitIO"])
            # Receive (RX) qubit on the port's input:
            message = self.node.ports["qubitIO"].rx_input()
            qubit = message.items[0]
            meas, prob = ns.qubits.measure(qubit, observable=self.observable)
            print(f"{ns.sim_time():5.1f}: {self.node.name} measured "
                  f"{self.basis[meas]} with probability {prob:.2f}")
            # Send (TX) qubit to the other node via connection:
            self.node.ports["qubitIO"].tx_output(qubit)

The constructor of this protocol (i.e. the __init__ method) can optionally be given a qubit, in which case it will use this qubit to kick the game off. The protocol is started with the start method, which will call run. The run method will run until the first yield where it will wait until it receives the qubit. Once the qubit is received the method continues; the qubit is measured and sent back. Once it is sent back the run method will encounter the yield once more and will wait again until it receives the qubit. This process repeats ad infinitum until one of the protocol stops.

In [15]:
# assign protocol to both nodes
qubits = ns.qubits.create_qubits(1)
ping_protocol = PingPongProtocol(node_ping, observable=ns.Z, qubit=qubits[0])
pong_protocol = PingPongProtocol(node_pong, observable=ns.X)

In [16]:
# running the simulation
ping_protocol.start()
pong_protocol.start()
run_stats = ns.sim_run(duration=300)

 17.6: Pong measured |+> with probability 0.50
 35.1: Ping measured |1> with probability 0.50
 53.8: Pong measured |+> with probability 0.50
 73.2: Ping measured |0> with probability 0.50
 93.1: Pong measured |+> with probability 0.50
110.6: Ping measured |1> with probability 0.50
128.6: Pong measured |+> with probability 0.50
146.8: Ping measured |1> with probability 0.50
165.8: Pong measured |-> with probability 0.50
184.3: Ping measured |0> with probability 0.50
204.5: Pong measured |+> with probability 0.50
222.9: Ping measured |0> with probability 0.50
241.4: Pong measured |+> with probability 0.50
259.7: Ping measured |1> with probability 0.50
278.3: Pong measured |-> with probability 0.50
298.0: Ping measured |1> with probability 0.50


In [18]:
# return useful diagnostics
print(run_stats)


Simulation summary

Elapsed wallclock time: 0:00:00.005126
Elapsed simulation time: 3.00e+02 [ns]
Triggered events: 32
Handled callbacks: 32
Total quantum operations: 16
Frequent quantum operations: MEASURE = 16
Max qstate size: 1 qubits
Mean qstate size: 1.00 qubits

